# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2021 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
It will be the same language game dataset as in exercise08.

1. Change to exercise09 repository

2. Start docker <br>
```docker-compose up -d```

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. 

More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. copy the data to docker :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to docker needs to be done only once and it might take 1-2 minutes.)

## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: http://lars.yencken.org/datasets/languagegame/

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [45]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise(field='sample'):
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))
    print(res)
    if type(res) == list:
        print(','.join([str(x[field]) for x in res]))


## <center>1. Spark Dataframes</center>

Write queries for the same questions as last week, but this time using Spark Dataframes operations (the data loading will take a minute)

### 1.0. Data preprocessing

In [2]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()

In [3]:
#test it out
dataset.limit(3).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
|[Danish, Dinka, K...|     AU|2013-08-19|    Dinka|af5e8f27cef9e689a...|    Dinka|
|[German, Hungaria...|     AU|2013-08-19|  Turkish|509c36eb58dbce009...|   Samoan|
+--------------------+-------+----------+---------+--------------------+---------+



In [8]:
import gc
gc.collect()

186

## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Maori.

In [20]:
start_exercise()

In [21]:
#Your code here
res = dataset.filter("target = guess and target = 'Maori'").count()
# res = dataset.filter("target = 'Maori' and target = guess").count()

In [22]:
finish_exercise()

This exercise took 4s
74810


## Assignment 2
Return the number of distinct "target" languages.

In [35]:
start_exercise()

In [36]:
#Your code here
res = dataset.select("target").distinct().count()

In [37]:
finish_exercise()

This exercise took 6s
78


## Assignment 3
Quiz version:
Return the sample IDs (i.e., the sample field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In order to help with the grading please add them in the following format: id1,id2,id3 (meaning separated with commas and without any spaces between them)

Notebook version:
Return the sample IDs (i.e., the *sample* field) of the bottom three games where the guessed language is incorrect (not equal to the target one) ordered by date (ascending), then by language (descending), then by country (ascending).

In [25]:
start_exercise()

In [26]:
#Your code here
res = dataset.filter("target = guess").orderBy(["target", "country", "date"], ascending=True).select("sample").limit(3).collect()
# res = dataset.select("sample").filter("guess = target").orderBy(dataset["target"].asc(), dataset["country"].asc(), dataset["date"].asc()).limit(3).collect()


In [27]:
finish_exercise()

This exercise took 8s
[Row(sample='00b85faa8b878a14f8781be334deb137'), Row(sample='efcd813daec1c836d9f030b30caa07ce'), Row(sample='efcd813daec1c836d9f030b30caa07ce')]
00b85faa8b878a14f8781be334deb137,efcd813daec1c836d9f030b30caa07ce,efcd813daec1c836d9f030b30caa07ce


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [46]:
start_exercise()

In [47]:
#Your code here
res = dataset.groupBy(["country", "target"]).count().orderBy("count", ascending=False).limit(3).collect()

In [48]:
finish_exercise("count")

This exercise took 8s
[Row(country='US', target='French', count=112934), Row(country='US', target='German', count=112007), Row(country='US', target='Spanish', count=110919)]
112934,112007,110919


## Assignment 5
Find the fraction (between 0 and 1) of games where (the answer was correct && the correct guess was the second choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3320)

In [52]:
start_exercise()

In [53]:
#Your code here
res = 1.0 * dataset.filter("target = choices[1] and target = guess").count() / dataset.count()
print(f'{res:.4f}')

0.2640


In [54]:
finish_exercise()

This exercise took 6s
0.26400199040361877


## Assignment 6
Sort the languages by increasing overall percentage of correct guesses and return the first three languages.

In [71]:
start_exercise()

In [72]:
#Your code here
correct_counts = dataset.filter("target = guess").groupBy("target").count().withColumnRenamed("count", "correct")
all_counts = dataset.groupBy("target").count().withColumnRenamed("count", "all")
tmp = correct_counts.join(all_counts, correct_counts["target"] == all_counts["target"])
res = tmp.withColumn("percentage", 1.0 * tmp["correct"] / tmp["all"]).orderBy("percentage").take(3)

del correct_counts, all_counts, tmp

In [73]:
finish_exercise("target")

This exercise took 11s
[Row(target='Kannada', correct=46923, target='Kannada', all=119509, percentage=0.39263151729158474), Row(target='Fijian', correct=48081, target='Fijian', all=115852, percentage=0.41502088872009113), Row(target='Shona', correct=36186, target='Shona', all=82478, percentage=0.43873517786561267)]
Kannada,Fijian,Shona


## Assignment 7
Return the number of games played on the first day.

In [74]:
start_exercise()

In [75]:
#Your code here
res = dataset.groupBy("date").count().orderBy("date", ascending=True).take(1)

In [76]:
finish_exercise("count")

This exercise took 5s
[Row(date='2013-08-19', count=163)]
163


## <center>2. Spark SQL</center>

Write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [77]:
!pip install sparksql-magic

     |████████████████████████████████| 198 kB 6.0 MB/s eta 0:00:01


In [89]:
  
def finish_exercise(field='sample'):
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))


In [120]:
%load_ext sparksql_magic

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [139]:
path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()
dataset.registerTempTable("dataset")

In [140]:
%%sparksql
-- test it out
SELECT *
FROM dataset
LIMIT 3

choices,country,date,guess,sample,target
"['Maori', 'Mandarin', 'Norwegian', 'Tongan']",AU,2013-08-19,Norwegian,48f9c924e0d98c959d8a6f1862b3ce9a,Norwegian
"['Danish', 'Dinka', 'Khmer', 'Lao']",AU,2013-08-19,Dinka,af5e8f27cef9e689a070b8814dcc02c3,Dinka
"['German', 'Hungarian', 'Samoan', 'Turkish']",AU,2013-08-19,Turkish,509c36eb58dbce009ccf93f375358d53,Samoan


## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Maori.

In [90]:
start_exercise()

In [91]:
%%sparksql
SELECT count(*)
FROM dataset
WHERE dataset.guess = dataset.target
AND dataset.target = "Maori";

count(1)
74810


In [92]:
finish_exercise()

This exercise took 4s


## Assignment 2
Return the number of distinct "target" languages.

In [94]:
start_exercise()

In [95]:
%%sparksql
SELECT count(distinct(dataset.target))
FROM dataset;

count(DISTINCT target)
78


In [96]:
finish_exercise()

This exercise took 5s


## Assignment 3
Return the sample IDs (i.e., the sample field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In order to help with the grading please add them in the following format: id1,id2,id3 (meaning separated with commas and without any spaces between them)

In [99]:
start_exercise()

In [100]:
%%sparksql
SELECT dataset.sample
FROM dataset
WHERE dataset.target == dataset.guess
ORDER BY dataset.target, dataset.country, dataset.date ASC
LIMIT 3;

sample
00b85faa8b878a14f8781be334deb137
efcd813daec1c836d9f030b30caa07ce
efcd813daec1c836d9f030b30caa07ce


In [101]:
finish_exercise()

This exercise took 9s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [104]:
start_exercise()

In [105]:
%%sparksql
SELECT count(dataset.country, dataset.target) as cnt
FROM dataset
GROUP BY dataset.country, dataset.target
ORDER BY cnt DESC
LIMIT 3;

cnt
112934
112007
110919


In [106]:
finish_exercise()

This exercise took 7s


## Assignment 5
Find the fraction (between 0 and 1) of games where (the answer was correct && the correct guess was the second choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3320)

In [115]:
start_exercise()

In [159]:
%%sparksql
SELECT (count(*) / (SELECT count(*) FROM dataset)) as frac
from dataset
WHERE target = choices[1] AND target = guess;

frac
0.26400199040361877


In [ ]:
finish_exercise()

## Assignment 6
Sort the languages by increasing overall percentage of correct guesses and return the first three languages.

In [ ]:
start_exercise()

In [153]:
%%sparksql

SELECT target
FROM (SELECT target, count(*) as correct_count
      FROM dataset
      WHERE target = guess
      GROUP BY target
    ) NATURAL JOIN (
      SELECT target, count(*) as all_count
      FROM dataset
      GROUP BY target
    )
ORDER BY (correct_count / all_count) ASC
LIMIT 3;

target
Kannada
Fijian
Shona


In [ ]:
finish_exercise()

## Assignment 7
Return the number of games played on the first day.

In [ ]:
start_exercise()

In [155]:
%%sparksql
SELECT date, count(*)
FROM dataset
GROUP BY date
ORDER BY date
LIMIT 1;

date,count(1)
2013-08-19,163


In [ ]:
finish_exercise()